[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA3/blob/main/lab6/lab6.ipynb)

# Lab6: Aprendizaje por refuerzo - Métodos de Temporal Difference

En este laboratorio seguiremos explorando métodos tabulares de aprendizaje por refuerzo que no necesitan disponer de un modelo. En particular, estudiaremos los métodos de **Temporal Difference**.

Para explorar estos métodos volveremos a utilizar [Gym](https://www.gymlibrary.dev/) con un entorno GridWorld, es decir, un tablero con casillas por las que se mueve el agente, tal como hicimos en el Lab4. En particular, vamos a utilizar el entorno `MiniGrid-DistShift1-v0`. En este entorno el agente deberá llegar hasta la meta, pero esta vez evitando caer en la lava, que hace que se termine el episodio con recompensa 0.

Carguemos el entorno y explorémoslo brevemente.

In [ ]:
# Si los paquetes no están instalados, hay que ejecutar estas líneas:
#!pip install gymnasium[classic-control]
#!pip install minigrid 
import gymnasium as gym
import minigrid
import numpy as np
env = gym.make('MiniGrid-DistShift1-v0', render_mode='rgb_array')

import matplotlib.pyplot as plt

# TODO - Muestra el entorno para ver cómo es el tablero. Para ello, recupera la función muestra_entorno del lab4 y lánzala
...

In [ ]:
# TODO - Anotar las dimensiones del tablero para poder utilizarlas después
NUM_COLUMNAS = ...
NUM_FILAS = ...
NUM_ORIENTACIONES = 4

# Las acciones son las mismas que en el laboratorio 4. Nos quedaremos de nuevo solo con LEFT, RIGHT y FORWARD
acciones = env.actions
# Seleccionamos solo las tres acciones indicadas
ACCIONES_UTILES = [acciones.left, acciones.right, acciones.forward]
NUM_ACCIONES = len(ACCIONES_UTILES)

Comprueba el efecto de caer en la lava. El episodio debería terminarse y recibiendo el agente una recompensa de 0.

In [ ]:
#TODO - Efectúa dos acciones para que el agente caiga en la lava y verifica que se recibe una recompensa de 0
obs, reward, terminated, truncated, info = ...
obs, reward, terminated, truncated, info = ...

#COMPROBACIONES
assert(reward==0)
assert(terminated)
_ = env.reset()

# Obtención de políticas óptimas

## Métodos de Temporal Difference

Los métodos basados en **Temporal Difference**, al igual que los métodos Montecarlo, carecen de información respecto a cómo funciona el modelo (no conocen $p(s',r | s,a)$). Por ello, también deben interactuar con el entorno para obtener muestras con las que después hacer estimaciones.

A diferencia de los métodos Montecarlo, los métodos de Temporal Difference no esperan a terminar un episodio para actualizar los valores de los estados transitados. En su lugar, los métodos Temporal Difference aprovechan la idea de que el valor $v(S)$ de un estado $S$ tiene relación con el valor de los estados $S'$ a los que se puede llegar desde $S$. Dicho de otra manera, los estados que pueden conducir a un estado malo serán también malos (y viceversa). Esta idea está codificada en las **ecuaciones de Bellman**:

$$v_\pi(s)=\sum_{a}\pi(a|s)\sum_{s',r} p(s',r | s,a)\left[r + \gamma v_\pi(s')\right]$$

La misma idea se puede aplicar a la hora de estimar el valor de ejecutar la acción $a$ estando en el estado $s$. Las acciones $a$ que lleven a estados $s'$ cuyas acciones tengan valores altos tendrán, a su vez, valores altos (y viceversa). 

$$q_\pi(s,a)=\sum_{s',r} p(s',r | s,a)\left[r + \gamma \sum_{a'}\pi(a'|s')q_\pi(s',a')\right]$$

Aplicar esta idea nos permitirá hacer *bootstrapping* en nuestros cálculos: a la hora de calcular el valor de un estado podremos aprovechar la aproximación que tenemos para estados adyacentes.

### SARSA
El primer método que probaremos es SARSA. Este algoritmo se denomina así por las variables involucradas en cada paso de actualización de la estimación de $Q(S,A)$. Las variables son $S_t,A_t,R_{t+1},S_{t+1},A_{t+1}$ y la actualización utiliza esta fórmula:

$$Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha [R_{t+1} + \gamma Q(S_{t+1},A_{t+1}) - Q(S_t,A_t)]$$

El algoritmo ejecutará repetidos episodios (potencialmente infinitos; es un algoritmo de control y el agente podría seguirlo utilizando durante toda su existencia) y, para cada paso del episodio hará la actualización según la fórmula indicada arriba. Esto requiere que haya decidido la siguiente acción a tomar ($A_{t+1}$) antes de actualizar el valor de la acción anterior ($A_t$).

![Sarsa](./img/sarsa.png)

Para facilitar la implementación, vamos primero a escribir un par de funciones auxiliares que nos permitan representar estados y muestrear acciones a partir de $Q$.

In [ ]:
GAMMA = 0.9 # Para este problema vamos a utilizar un factor de descuento de 0.9, es decir, las recompensas futuras se descontarán un 10% por cada paso que sea necesario para obtenerlas.

env.max_steps = 5000 # Fijamos el número máximo de acciones por episodio a 5000, para permitir episodios largos

# FUNCIONES AUXILIARES
#TODO - Recupera la función get_estado del laboratorio 4. Esta función codifica el estado del entorno en una lista de tres elementos: columna, fila, orientación
def get_estado(env:gym.Env) -> Estado:
...

# COMPROBACIÓN
env.reset()
estado_actual = get_estado(env)
assert estado_actual.x == 0, f'El estado inmediatamente después de resetear debe indicar x=0 y el tuyo indica {estado_actual.x}'
assert estado_actual.y == 0, f'El estado inmediatamente después de resetear debe indicar y=0 y el tuyo indica {estado_actual.y}'
assert estado_actual.dir == 0, f'El estado inmediatamente después de resetear debe indicar dir=0 y el tuyo indica {estado_actual.dir}'

# TODO - Escribe una función que dados los valores q de las distintas acciones en un estado concreto, devuelva una acción haciendo una selección epsilon-greedy a partir de los q_valores.
def get_accion_epsilon_greedy(q_valores:np.ndarray, epsilon:float):
    '''
    Selecciona una acción basándose en los q_values proporcionados. Una fracción de las veces (indicada por epsilon) devolverá una acción al azar
    
    Argumentos:
    q_values -- Lista con los q valores de las acciones entre las que seleccionar.
    epsilon -- Fracción de las veces que se devolverá una acción al azar
    '''
    if np.random.random()<epsilon:
        # TODO - Devuelve una acción al azar
        return ...
    # TODO - Devuelve el índice del mayor q valor
    return ...


# COMPROBACIÓN
assert(get_accion_epsilon_greedy([1,2,4],0)==2)

Para algoritmos derivados de SARSA haremos _exploring starts_, es decir, comienzarán cada vez en un estado al azar. Esto facilita la exploración y, por tanto, el entrenamiento ya que facilita que se den episodios que el agente sea capaz de resolver incluso con una política aleatoria.

Deberemos crear funciones que nos permitan llevar esto a cabo.

In [ ]:
# TODO - Escribe una función que genere la codificación para un estado válido al azar. Devolverá una lista con tres valores que indiquen columna, fila y orientación
def get_estado_aleatorio() -> Estado:
    ...

# TODO - Escribe una función que establezca el entorno para que coincida con lo indicado por un estado dado
def set_estado(env:gym.Env, estado: Estado):
    ...

# COMPROBACIÓN
env.reset()
muestra_entorno(env) # Debe mostrar al agente en la casilla 0,0
set_estado(env, Estado(x=2, y=3, dir=0))
muestra_entorno(env) # Debe mostrar al agente en la casilla 2,3

Ya tenemos lo necesario para implementar SARSA. Vamos a establecer un número fijo de iteraciones y hacer que devuelva los valores $Q$ al terminar. Además, incluiremos mensajes al terminar cada episodio que indiquen:
 - El número de episodio que se ha finalizado
 - La recompensa obtenida
 - El número de acciones necesarias para completar el episodio
 - La recompensa media obtenida durante los últimos 100 episodios (si se han ejecutado al menos 100)

In [ ]:
def sarsa(num_episodios:int = 500, ALPHA:float = 0.1, EPSILON:float = 0.25) -> np.ndarray:
    # TODO - Inicializa los valores Q a cero con el shape adecuado
    q_valores = ...
    
    # En ultimos100retornos almacenaremos los retornos de los últimos 100 episodios
    ultimos100retornos = []
    # TODO - Haz tantos episodios como indica num_episodes
        # TODO - Devuelve el entorno a su estado inicia
        # TODO - Initialize S
        # TODO - Choose A from S using policy derived from Q (e.g. epsilon-greedy)
        
        retorno = 0
        num_pasos = 0
        # TODO - Loop for each step of episode (repite hasta que el episodio termine)
            num_pasos += 1
            # TODO - Take action A, observe R, S'
            # TODO - Acumula la recompensa R a returns para poder imprimir el retorno del episodio
            # TODO - Choose A' from S' using policy derived from Q (e.g. epsilon-greedy)
            # TODO - Q(S,A) <- Q(S,A) + ALPHA * [R + GAMMA * Q(S',A') - Q(S,A)]
            # TODO - S <- S'; A <- A'
            
        # Tras terminar el episodio almacenamos el retorno en los 100 últimos...
        ultimos100retornos.append(retorno)
        string_retornos100 = ''
        if len(ultimos100retornos)==101: # ... y si hay ya más de 100...
            ultimos100retornos.pop(0) # ...quitamos el más antiguo (para tener siempre 100, no más)...
            #... y preparamos el mensaje para mostrar la media de los retornos
            string_retornos100=f'(retorno medio de {np.mean(ultimos100retornos)} en los últimos 100 episodios)'
        # Mostramos el mensaje tras cada episodio
        print(f'Terminado episodio {i} con retorno {retorno} en {num_pasos} pasos {string_retornos100}')
    return q_valores

# Entrenamos al agente usando los parámetros por defecto de la función que acabamos de definir
q_valores_sarsa = sarsa()

El proceso de aprendizaje tiene una componente aleatoria importante, por lo que dos ejecuciones consecutivas pueden aprender valores $Q$ diferentes de las que se deriven políticas $\pi$ también diferentes. No obstante, si has implementado bien el algoritmo, deberías haber observado lo siguiente:
  1. Inicialmente hay muchos episodios con retorno 0 (el agente acaba en la lava). La duración de estos episodios es muy variable. Además, si imprimiésemos `q_values` tras cada uno de estos episodios, ¡veríamos que $Q$ no se modifica! Cada paso utiliza $R$ y $Q(S',A')$ para actualizar $Q(S,A)$, pero si tanto $R$ como $Q(S',A')$ son cero, el valor de $Q(S,A)$ no cambiará.
  1. Con el tiempo, y por pura casualidad, el agente terminará algún episodio llegando a la meta. Esto le reportará una recompensa de 1, lo que hará que $Q(S_t,A_t)$ se actualice a un valor positivo.
  1. La próxima vez que el agente esté en $S_t$, elegirá la acción $A_t$ (salvo que toque acción aleatoria por el $\epsilon$), lo que hará que termine el episodio con recompensa 1. **¡Habrá aprendido una política útil para la última casilla!** Pero, además, el estado $S_{t-1}$ que llevó a $S_t$ (usando la acción $A_{t-1}$) verá su $Q(S_{t-1},A_{t-1})$ actualizado a un valor positivo. **¡Ya sabrá una política útil para las dos últimas casillas!**
  1. Mediante este proceso, la información respecto a qué política es útil se irá propagando desde las casillas aledañas a la meta a sus vecinas, y de estas a sus vecinas y así sucesivamente hasta cubrir el tablero entero. A esto hay que añadir que, también por pura casualidad, el agente puede encontrar otros caminos que conducen a la meta y lanzar este mismo proceso de propagación pero siguiendo otro camino que conduzca a la meta.
  1. El efecto de esto es que, a medida que transcurren los episodios ocurren dos cosas:
    - El número de episodios con retorno positivo aumenta
    - La duración de los episodios con retorno positivos disminuye (el agente va más tiempo por "camino conocido")
  1. Esto repercute en el retorno medio de los 100 últimos episodios: comenzará siendo minúsculo, pero llegado un punto subirá con gran velocidad.
  1. El retorno de los últimos 100 episodios alcanzará un punto máximo en torno al cual oscilará hacia el final.
  
  
Visualicemos la política aprendida. Vamos a mostrar, para cada casilla, la orientación que tiene el agente cuando la acción preferida (la que tiene mayor valor $Q$) sea `FORWARD`.

>**Recordatorio**
>
> Las orientaciones se codifican así:
> - 0 $\rightarrow$ derecha
> - 1 $\rightarrow$ abajo
> - 2 $\rightarrow$ izquierda
> - 3 $\rightarrow$ arriba

In [ ]:
# Esta función devuelve un carácter de flecha que representa la orientación del agente para la cual los q valores recomiendan la acción forward
def get_flecha_direccion_maximo_valor(q_values:np.ndarray) -> str:
    accion_preferida_por_orientacion = q_values.argmax(axis=1) # Para cada una de las cuatro orientaciones, calculamos qué acción se prefiere
    direccion = np.argmax(accion_preferida_por_orientacion == 2) # De las orientaciones que prefieran FORWARD, tomamos una (la primera)
    # En función de la orientación, devolvemos la flecha apropiada.
    if direccion==0:
        return '⮕'
    if direccion==1:
        return '⬇'
    if direccion==2:
        return '⬅'
    if direccion==3:
        return '⬆'
    return '?'
    
# Muestra la política por pantalla
def dibuja_politica(q_valores:np.ndarray) -> None:
    # Recorre las filas del tablero...
    for i in range(q_valores.shape[1]):
        # ... componiendo una línea por fila...
        linea = ''
        for j in range(q_valores.shape[0]): # Recorre las columnas
            if i==0 and j==6: # Pintamos la meta
                    linea+='🟥'
            elif (i==0 or i==1)and j>1 and j<5: # Pintamos la lava
                    linea+='🟧'
            else:
                linea+=get_flecha_direccion_maximo_valor(q_valores[j,i]) # Pintamos la flecha apropiada para esta casilla
        # ...que finalmente imprime
        print(linea)
        
# TODO - Muestra la política que se deriva de los valores aprendidos por SARSA
...

Hemos comprobado que nuestro agente tiene que hacer muchos episodios que no le llevan a aprender porque no le dan ninguna recompensa, lo cual provoca que no pueda actualizar $Q(S,A)$ para ninguno de los pares $S,A$ por los que transita.

**¿Qué crees que ocurriría si diésemos una recompensa negativa a caer en la lava? ¿Cambiaría la velocidad a la que aprende el agente?**

Haz la prueba añadiendo esto después de cada paso:
```python
    if completado and reward==0:
        reward = -1
```

Modifica la función `sarsa` y ejecuta la celda siguiente. **¿Qué observas?**

In [ ]:
q_valores_sarsa_con_lava_negativa = sarsa() # Sarsa debe haberse modificado para que caer a la lava proporcione recompensa -1
dibuja_politica(q_valores_sarsa_con_lava_negativa)

### Q-learning
El algoritmo Q-learning es de los más populares dentro del Aprendizaje por Refuerzo. Es muy similar a SARSA, pero tiene una importante diferencia: es *off-policy*. Si SARSA hacía las actualizaciones de $Q(S_t,A_t)$ en función de la acción $A_{t+1}$ que tomaba, Q-learning va a hacer dichas actualizaciones independientemente de la acción que finalmente tome. Por tanto, estará utilizando una política para explorar pero otra para actualizar. En concreto, Q-learning actualiza según esta fórmula:

$$Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha [R_{t+1} + \gamma \max_aQ(S_{t+1},a) - Q(S_t,A_t)]$$

Q-learning no actualiza en función de la acción que va a tomar, sino en función de la mejor acción posible. El algoritmo completo aparece descrito a continuación.

![Q-learning](./img/q-learning.png)

Implementemos Q-learning mostrando los mismos mensajes a cada paso que mostrábamos con SARSA. Mantén las recompensas negativas para cuando el agente caiga en la lava.

In [ ]:
# TODO - Implementa q_learning
#  Utiliza el mismo código que para sarsa pero cambiando la actualización
#  Recuerda mantener las recompensas negativas para cuando el agente caiga en la lava y mostrar los mensajes
def q_learning(num_episodios:int = 500, ALPHA:float = 0.1, EPSILON:float = 0.25) -> np.ndarray:
    ...

q_valores_q = q_learning()
dibuja_politica(q_valores_q)

### Comparativa SARSA vs Q-Learning
Compara los resultados obtenidos por ambos algoritmos. Deberías observar lo siguiente:
  - La política obtenida por SARSA es más conservadora (no quiere estar cerca de la lava), mientras que la de Q-learning es más optimista (no le importa estar cerca de la lava; confía en su política).
  - Por tanto, los episodios de Q-learning son más cortos que los de SARSA.
  - Sin embargo, al seguir una política $\epsilon$-greedy, estar en una casilla contigua a la lava desemboca en una recompensa de -1 (caer a la lava) un $\frac{\epsilon}{4}$ de las veces, por lo que el retorno medio de Q-learning es más bajo.
  
Completa la siguiente celda para comprobarlo.

In [ ]:
def comprueba_politica_derivada(q_values:np.ndarray, epsilon:float) -> None:
    env.reset()
    # TODO - Escribe un bucle que simule un episodio completo siguiendo una política COMPLETAMENTE GREEDY respecto a q_values
    # y muestra su duración y su retorno
    ...
    print(f'La política greedy hace un episodio de {contador} pasos con un retorno de {retorno}')
    
    env.reset()
    # TODO - Escribe otro bucle que simule 500 episodios siguiendo una política epsilon-greedy respecto a q_values
    # y muestra su duración media y su retorno medio
    ...    
    print(f'La política greedy hace episodios de {np.mean(duraciones)} pasos de media con un retorno de {np.mean(retornos)} de media')

print('SARSA')
comprueba_politica_derivada(q_valores_sarsa_con_lava_negativa, 0.25)
dibuja_politica(q_valores_sarsa_con_lava_negativa)
print('\nQ-LEARNING')
comprueba_politica_derivada(q_valores_q, 0.25)
dibuja_politica(q_valores_q)